# Тема 3. Языковые модели и тематическое моделирование

1) N-граммы  
2) Языковые модели и перплексия  
3) Тематическое моделирование

### 1) N-граммы

На прошлом занятии мы познакомились с базовыми техниками работы с текстами, в т.ч. токенизацией на отдельные слова. Однако в естественных языках существуют устойчивые комбинации слов (словосочетания), несущие в себе собственный смысл, например ice cream или доброе утро.  
**Коллокация (collocation)** — словосочетание, имеющее признаки синтаксически и семантически целостной единицы.
К коллокациям также обычно причисляют составные топонимы, антропонимы и другие часто совместно употребляемые именования (например, крейсер «Аврора», завод имени Кирова).
В NLP пары слов называются биграммами, тройки — триграммами, и т.д..  
**N-грамма** — это последовательность, содержащая до n элементов, которые были извлечены из последовательности этих элементов, обычно строки. Использование n-грамм позволяет машине знать о таких словах, как ice cream, а также о составляющих его ice и cream. 
В общем случае элементами n-граммы могут быть буквы, слоги, слова или даже символы, такие как A, T, G и C, используемые для представления последовательности ДНК. В дальнейшем под N-граммой мы будем понимать только комбинацию слов.  

Зачем нужны на n-граммы? При преобразовании в OneHot вектор слов последовательность токенов теряет часть смысла, заключенного в порядке этих слов. Если расширить концепцию токена на токены из нескольких слов (n-грамм), то можно сохранить значительную долю смысла, заключенного в порядке слов в этих высказываниях. Например, слово not, меняющее смысл на обратный, останется рядом с соседними словами, где и должно быть. Без n-граммовой токенизации такое слово болталось бы по разным позициям, а его смысл связывался бы со всем предложением или документом, а не с соседними словами. Биграмма was not сохраняет гораздо больше смысла отдельных слов was и not, чем соответствующие однограммы в векторе OneHot. Таким образом, сохраняется небольшая часть контекста слова. 

In [1]:
from nltk.util import ngrams
sent = 'Thomas Jefferson began building Monticello at the age of 26'
tokens_2 = ngrams(sent.split(), 2)
for t in tokens_2:
    print(t)

('Thomas', 'Jefferson')
('Jefferson', 'began')
('began', 'building')
('building', 'Monticello')
('Monticello', 'at')
('at', 'the')
('the', 'age')
('age', 'of')
('of', '26')


Можно предположить, что токен "Thomas Jefferson" будет часто встречаться во многих документах, а "of 26" или "Jefferson began" редко. 
Большинство биграмм довольно редки, не говоря уже о 3- и 4-граммах. 
Необходимо использовать статистику частотностей n-грамм, и оставлять только часто встречающиеся. 
Как правило, слишком редкие n-граммы отфильтровываются, а часто встречающиеся включаются в словарь наравне со словами. 

Противоположная проблема — биграмма "at the" из примера. 
Это нередкая комбинация слов, но она непригодна для различения смысла документа. 
Подобно словам и другим токенам, слишком часто встречающиеся n-граммы отфильтровываются. 
Токены или n-граммы, встречающиеся в более чем 25 % всех документов в корпусе, обычно игнорируются. 
Использование подобных фильтров полезно как в отношении n-грамм, так и для отдельных токенов.

Для оценки частотности слов используется техника Bag Of Words

In [2]:
from nltk.tokenize import word_tokenize
from collections import Counter

sentence = 'John likes to watch movies, Mary likes movies too.'

tokens = word_tokenize(sentence)
bow = Counter(tokens)

print(f"Bag of words:\n{bow}")

Bag of words:
Counter({'likes': 2, 'movies': 2, 'John': 1, 'to': 1, 'watch': 1, ',': 1, 'Mary': 1, 'too': 1, '.': 1})


### 2) Языковые модели и перплексия

Рассмотрим две фразы: "Студенты пишут конспекты" и "Студенты пишут романы". Если мы возьмем довольно большой корпус текста (например, все произведения русской литературы), вероятность встретить там первую фразу будет выше, чем вторую.  

**Языковая модель** - это распределение вероятностей по всевозможным последовательностям слов.  

Для последовательности слов длиной m, языковая модель позволяет оценить вероятность того, что эта последовательность встречается в тексте.  

$$p_{sent} = P(t_{1},\ldots , t_{m})$$  

Простая униграмная модель не учитывает взаимное расположение слов. 

$$P_{\text{uni}}(t_{1}t_{2}t_{3})=P(t_{1})P(t_{2})P(t_{3})$$

Чтобы модель запомнила порядок слов, можно просто посчитать количество всех токенов, потом биграмм, триграмм и т.д., и сохранить эти значения. Например, биграмная модель вычисляет вероятность по формуле:  

$$P_{\text{bi}}(t_{1}t_{2}t_{3})=P(t_{1}|t_2)P(t_{2}|t_{3})$$

Однако по мере увеличения объема корпуса, необходимая память для такой модели растет экспоненциально. 

На практике языковые модели генерируют вероятности путем *обучения* на больших текстовых корпусах, с использованием цепей Маркова, рекуррентных нейросетей или трансформеров. Продвинутые модели дают ненулевую вероятность даже для тех семантически и синтаксически допустимых последовательностей, которых не было в обучающей выборке. 

Более простая задача для языковой модели - предсказание следующего слова в заданном контексте. 

$$P(t_{n}|t_{n-1}, .. t_{1})$$

Например, такую работу выполняет клавиатура при наборе сообщений на мобильном телефоне, предлагая три самых вероятных слова.  

Распределение вероятности, предсказанное моделью, и распределение слов в корпусе, отличаются друг от друга. Если в тексте встречались фразы "студенты пишут картины", "студенты пишут зачет", "студенты пишут конспекты" (2 раза)  то распределение вероятностей последнего слова в фразе будет {'картины' - 0.25, 'зачет'-0.25, 'конспекты'-0.5}. Предсказанные вероятности для хорошей и плохой языковой модели представлены на рисунке.  

<img src='imgs/lm.png'>


Для того, чтобы оценить, насколько разные распределения вероятностей похожи друг на друга, используют понятие кросс-энтроии.

$$\mathrm {H} (p,q)=-\sum _{x}p(x)\,\log q(x)$$

где $p$ и $q$ - два разных распределения.

**Перплексия** языковой модели  — это способ оценки качества языковых моделей, вычисляется как 2 в степени энтропии.  

$$PP(p):=2^{H(p,q)}=2^{-\sum _{x}p(x)\log _{2}q(x)}$$


Перплексия может использоваться для сравнения качества статистических моделей. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

In [3]:
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline

lm = MLE(2)
text = ['студенты пишут картины'.split(),
        'студенты пишут зачет'.split(),
        'студенты пишут конспекты'.split(),
        'студенты пишут программы'.split()]

train, vocab = padded_everygram_pipeline(2, text)
lm.fit(train, vocab)

In [4]:
lm.score('картины')

0.05

In [5]:
lm.score('пишут', ['студенты'])

1.0

### 3) Тематическое моделирование

Знание того, о чем пишут люди, и понимание их проблем и мнений очень ценно для бизнеса. Очень трудно а иногда не возможно прочитать большие объемы теста человеком. Поэтому, требуется автоматизированный алгоритм, который может читать текстовые документы и автоматически выводить темы из текста.  

**Тематическое моделирование** — способ построения модели коллекции текстовых документов, которая определяет, к каким темам относится каждый из документов.  
**Тематическая модель** (topic model) коллекции текстовых документов определяет, к каким темам относится каждый документ и какие слова (термины) образуют каждую тему.  

Интуитивно понимая, что документ относится к определённой теме, в документах, посвящённых одной теме, можно встретить некоторые слова чаще других. Например: «собака» и «кость» встречаются чаще в документах про собак, «кошки» и «молоко» будут встречаться в документах о котятах, предлоги «и» и «в» будут встречаться в обеих тематиках. Обычно документ касается нескольких тем в разных пропорциях, таким образом, документ в котором 10 % темы составляют кошки, а 90 % темы — собаки, можно предположить, что слов про собак в 9 раз больше. Тематическое моделирование отражает эту интуицию в математической структуре, которая позволяет на основании изучения коллекции документов и исследования частотных характеристик слов в каждом документе сделать вывод, что каждый документ — это некоторый баланс тем.

In [6]:
pip install gensim, spacy

ERROR: Invalid requirement: 'gensim,'
Note: you may need to restart the kernel to use updated packages.


In [7]:
! pyton -m spacy download ru_core_news_sm

/bin/bash: pyton: command not found


In [11]:
import spacy
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import gensim.corpora as corpora
import gensim.models as gm

nlp = spacy.load("ru_core_news_sm")

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def remove_stopwords(texts, stop_words):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [12]:
with open('texts/vk.txt') as f:
    corpus = f.read().splitlines()

sp_corpus = [simple_preprocess(doc) for doc in corpus if len(doc)>100]

lemm_sp_corp = lemmatization(sp_corpus)

ru_stopwords = stopwords.words("russian")
ru_stopwords += ['так', 'уже', 'очень', 'муж']
clear_corpus= remove_stopwords(lemm_sp_corp, ru_stopwords)

id2word = corpora.Dictionary(clear_corpus)

ids_corpus = [id2word.doc2bow(text) for text in clear_corpus]

lda_model = gm.ldamodel.LdaModel(corpus=ids_corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model.print_topics()

[(0,
  '0.013*"мужчина" + 0.008*"мама" + 0.008*"дело" + 0.006*"давать" + 0.006*"нужный" + 0.006*"наблюдать" + 0.006*"должный" + 0.006*"сама" + 0.006*"беременность" + 0.006*"отец"'),
 (1,
  '0.011*"девушка" + 0.009*"отношение" + 0.009*"тыс" + 0.009*"рубль" + 0.009*"бывший" + 0.007*"мужик" + 0.007*"ребёнок" + 0.007*"знакомая" + 0.007*"говорить" + 0.007*"давать"'),
 (2,
  '0.012*"семья" + 0.012*"любить" + 0.010*"ребёнок" + 0.010*"хотеть" + 0.008*"говорить" + 0.008*"время" + 0.008*"мочь" + 0.008*"коллега" + 0.008*"деньга" + 0.008*"гость"'),
 (3,
  '0.026*"ребёнок" + 0.010*"девушка" + 0.010*"глаз" + 0.007*"бывший" + 0.007*"вопрос" + 0.007*"секс" + 0.007*"год" + 0.007*"получить" + 0.007*"жена" + 0.005*"женщина"')]

### Задание:
Подберите свой корпус текста для тематического моделирования.

- https://webdevblog.ru/tematicheskoe-modelirovanie-s-pomoshhju-gensim-python/